In [1]:
import pandas as pd
import numpy as np
import copy
import os
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt

## For case studies

In [3]:
benchmark_directory = "/projects/mcmm/results_final/"

In [3]:
benchmark_directory = "/home/ubuntu/s3/results_final/"

In [4]:
def get_info_from_filename(np_fname):
    splitted = np_fname.lower().split("_")
    func = splitted[0]
    dim = int(splitted[1])
    
    algorithm = None
    if "basinhopping" in np_fname:
        assert algorithm is None; algorithm = "basinhopping"
    elif "differential_evolution" in np_fname:
        assert algorithm is None; algorithm = "differential_evolution"
    elif "direct" in np_fname:
        assert algorithm is None; algorithm = "direct"
    elif "dual_annealing" in np_fname:
        assert algorithm is None; algorithm = "dual_annealing"
    elif "shgo" in np_fname:
        assert algorithm is None; algorithm = "shgo"
    elif "simulated_annealing" in np_fname:
        assert algorithm is None; algorithm = "simulated_annealing"
    elif "mcir" in np_fname:
        assert algorithm is None; algorithm = "mcir"
    elif "lamcts" in np_fname:
        assert algorithm is None; algorithm = "lamcts"
    elif "cma" in np_fname:
        assert algorithm is None; algorithm = "cma"
    elif "turbo" in np_fname:
        assert algorithm is None; algorithm = "turbo"
    
    assert algorithm is not None, "algorithm not found in {}".format(np_fname)
    
    info = {
        "function": func,
        "dimension": dim,
        "algorithm": algorithm,
    }
    return info

In [5]:
all_results = []

for filename in os.listdir(benchmark_directory):
    if os.path.isdir(os.path.join(benchmark_directory, filename)):
        continue
    assert filename.endswith("_Y.npy"), "Filename {} does not end with _Y.npy".format(filename)
    result_dict = get_info_from_filename(filename)
    result_dict['obj_list'] = np.load(os.path.join(benchmark_directory, filename))
    all_results.append(result_dict)

In [6]:
# Generate dataframe for results

In [7]:
def process_list(nums):
    result = []
    min_num = float('inf')  # initialize minimum value to positive infinity
    for num in nums:
        if num < min_num:
            min_num = num
        result.append(min_num)
    return result

In [8]:
result_df = pd.DataFrame(all_results)
result_df['min_found_list'] = result_df['obj_list'].apply(process_list)
result_df = result_df.sort_values(["function", "dimension", "algorithm"])
result_df.to_pickle("raw_result_df.pkl")
result_df

,function,dimension,algorithm,obj_list,min_found_list
187,ackley,10,basinhopping,"[20.554607, 20.554607, 20.554607, 20.554607, 2...","[20.554607, 20.554607, 20.554607, 20.554607, 2..."
358,ackley,10,basinhopping,"[21.551155, 21.551155, 21.551155, 21.551155, 2...","[21.551155, 21.551155, 21.551155, 21.551155, 2..."
490,ackley,10,basinhopping,"[21.23412, 21.23412, 21.23412, 21.23412, 21.23...","[21.23412, 21.23412, 21.23412, 21.23412, 21.23..."
577,ackley,10,basinhopping,"[21.037682, 21.037682, 21.037682, 21.037682, 2...","[21.037682, 21.037682, 21.037682, 21.037682, 2..."
709,ackley,10,basinhopping,"[20.38853, 20.38853, 20.38853, 20.38853, 20.38...","[20.38853, 20.38853, 20.38853, 20.38853, 20.38..."
...,...,...,...,...,...
462,watson,32,turbo,"[1279958.2190289465, 2967330.7083483934, 46156...","[1279958.2190289465, 1279958.2190289465, 46156..."
588,watson,32,turbo,"[15375177.685017385, 1235135.7184744407, 10396...","[15375177.685017385, 1235135.7184744407, 10396..."
591,watson,32,turbo,"[1121891.010572364, 2180993.1654268927, 332895...","[1121891.010572364, 1121891.010572364, 1121891..."
703,watson,32,turbo,"[369226.6069404025, 344591.4692395769, 321832....","[369226.6069404025, 344591.4692395769, 321832...."


In [9]:
result_df[result_df['algorithm'] == "lamcts"]

,function,dimension,algorithm,obj_list,min_found_list
168,ackley,50,lamcts,"[21.437702178955078, 21.103546142578125, 21.23...","[21.437702178955078, 21.103546142578125, 21.10..."
232,ackley,50,lamcts,"[21.44068145751953, 21.380264282226562, 21.092...","[21.44068145751953, 21.380264282226562, 21.092..."
559,ackley,50,lamcts,"[21.413639068603516, 21.129638671875, 21.26558...","[21.413639068603516, 21.129638671875, 21.12963..."
583,ackley,50,lamcts,"[20.915443420410156, 21.383880615234375, 21.53...","[20.915443420410156, 20.915443420410156, 20.91..."
687,ackley,50,lamcts,"[21.182109832763672, 21.08633804321289, 21.241...","[21.182109832763672, 21.08633804321289, 21.086..."
...,...,...,...,...,...
97,watson,32,lamcts,"[4226062.596401818, 1520741.3544014867, 48223....","[4226062.596401818, 1520741.3544014867, 48223...."
152,watson,32,lamcts,"[671131.1895748795, 4732821.699807334, 55126.3...","[671131.1895748795, 671131.1895748795, 55126.3..."
382,watson,32,lamcts,"[7942064.366904288, 794566.5601174604, 5912031...","[7942064.366904288, 794566.5601174604, 794566...."
460,watson,32,lamcts,"[910180.7320899942, 38635.83622881685, 63993.5...","[910180.7320899942, 38635.83622881685, 38635.8..."


In [10]:
result_df[result_df['algorithm'] == "cma"]

,function,dimension,algorithm,obj_list,min_found_list
427,ackley,50,cma,"[5.832585334777832, 6.885639190673828, 5.73112...","[5.832585334777832, 5.832585334777832, 5.73112..."
439,ackley,50,cma,"[6.3537492752075195, 5.679329872131348, 6.0435...","[6.3537492752075195, 5.679329872131348, 5.6793..."
471,ackley,50,cma,"[5.7271623611450195, 6.519017219543457, 5.0261...","[5.7271623611450195, 5.7271623611450195, 5.026..."
604,ackley,50,cma,"[6.157937049865723, 5.541773796081543, 5.81373...","[6.157937049865723, 5.541773796081543, 5.54177..."
873,ackley,50,cma,"[6.056830406188965, 6.073685646057129, 5.51450...","[6.056830406188965, 6.056830406188965, 5.51450..."
...,...,...,...,...,...
29,watson,32,cma,"[15882.364842876415, 20100.050695436796, 18311...","[15882.364842876415, 15882.364842876415, 15882..."
239,watson,32,cma,"[7636.070150666554, 15316.013655721252, 15621....","[7636.070150666554, 7636.070150666554, 7636.07..."
654,watson,32,cma,"[38722.327756693274, 18439.452079080816, 10887...","[38722.327756693274, 18439.452079080816, 18439..."
663,watson,32,cma,"[10384.85824696464, 5409.130540857736, 19874.5...","[10384.85824696464, 5409.130540857736, 5409.13..."


In [11]:
result_df[result_df['algorithm'] == "turbo"]

,function,dimension,algorithm,obj_list,min_found_list
411,ackley,50,turbo,"[21.32453155517578, 21.295059204101562, 21.197...","[21.32453155517578, 21.295059204101562, 21.197..."
494,ackley,50,turbo,"[21.129322052001953, 21.35611343383789, 21.140...","[21.129322052001953, 21.129322052001953, 21.12..."
659,ackley,50,turbo,"[21.566390991210938, 21.14297866821289, 21.267...","[21.566390991210938, 21.14297866821289, 21.142..."
689,ackley,50,turbo,"[21.136478424072266, 21.15451431274414, 21.322...","[21.136478424072266, 21.136478424072266, 21.13..."
766,ackley,50,turbo,"[21.031330108642578, 21.309558868408203, 21.33...","[21.031330108642578, 21.031330108642578, 21.03..."
...,...,...,...,...,...
462,watson,32,turbo,"[1279958.2190289465, 2967330.7083483934, 46156...","[1279958.2190289465, 1279958.2190289465, 46156..."
588,watson,32,turbo,"[15375177.685017385, 1235135.7184744407, 10396...","[15375177.685017385, 1235135.7184744407, 10396..."
591,watson,32,turbo,"[1121891.010572364, 2180993.1654268927, 332895...","[1121891.010572364, 1121891.010572364, 1121891..."
703,watson,32,turbo,"[369226.6069404025, 344591.4692395769, 321832....","[369226.6069404025, 344591.4692395769, 321832...."


In [12]:
# Generate statistics for results

In [13]:
stat_df = result_df.groupby(['function', 'dimension', 'algorithm']).agg(
    obj_lists=('obj_list', list),
    min_found_lists=('min_found_list', list)
).reset_index()
stat_df

,function,dimension,algorithm,obj_lists,min_found_lists
0,ackley,10,basinhopping,"[[20.554607, 20.554607, 20.554607, 20.554607, ...","[[20.554607, 20.554607, 20.554607, 20.554607, ..."
1,ackley,10,differential_evolution,"[[21.52509, 21.354877, 21.236618, 21.130295, 2...","[[21.52509, 21.354877, 21.236618, 21.130295, 2..."
2,ackley,10,direct,"[[4.552539, 17.018728, 17.45362, 17.018728, 17...","[[4.552539, 4.552539, 4.552539, 4.552539, 4.55..."
3,ackley,10,dual_annealing,"[[20.38853, 21.0765, 21.456715, 21.273575, 21....","[[20.38853, 20.38853, 20.38853, 20.38853, 20.3..."
4,ackley,10,mcir,"[[20.974483489990234, 20.974483489990234, 20.9...","[[20.974483489990234, 20.974483489990234, 20.9..."
...,...,...,...,...,...
165,watson,32,dual_annealing,"[[5393798.4407911245, 1104908.5937188813, 3621...","[[5393798.4407911245, 1104908.5937188813, 1104..."
166,watson,32,lamcts,"[[4226062.596401818, 1520741.3544014867, 48223...","[[4226062.596401818, 1520741.3544014867, 48223..."
167,watson,32,mcir,"[[651574.6720559866, 651574.6720559866, 651574...","[[651574.6720559866, 651574.6720559866, 651574..."
168,watson,32,shgo,"[[], [], [], [], []]","[[], [], [], [], []]"


In [14]:
def calculate_mean_and_std_of_lists(lists):
    pd_lists = pd.DataFrame(lists)
    means = pd_lists.mean(axis = 0).to_numpy()
    std = pd_lists.std(axis=0).fillna(0).to_numpy()
    
    return {
        "avg_min_found": means,
        "std_min_found": std
    }

In [15]:
from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
applied_df = stat_df.parallel_apply(
    lambda row: calculate_mean_and_std_of_lists(row.min_found_lists), 
    axis='columns', 
    result_type='expand'
)
stat_df = pd.concat([stat_df, applied_df], axis='columns')

In [17]:
stat_df.to_pickle("result_df.pkl")
stat_df

,function,dimension,algorithm,obj_lists,min_found_lists,avg_min_found,std_min_found
0,ackley,10,basinhopping,"[[20.554607, 20.554607, 20.554607, 20.554607, ...","[[20.554607, 20.554607, 20.554607, 20.554607, ...","[20.95322036743164, 20.95322036743164, 20.9532...","[0.4799298346042633, 0.4799298346042633, 0.479..."
1,ackley,10,differential_evolution,"[[21.52509, 21.354877, 21.236618, 21.130295, 2...","[[21.52509, 21.354877, 21.236618, 21.130295, 2...","[21.358928680419922, 21.04205322265625, 20.861...","[0.23468589782714844, 0.45797276496887207, 0.3..."
2,ackley,10,direct,"[[4.552539, 17.018728, 17.45362, 17.018728, 17...","[[4.552539, 4.552539, 4.552539, 4.552539, 4.55...","[4.552539, 4.552539, 4.552539, 4.552539, 4.552...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,ackley,10,dual_annealing,"[[20.38853, 21.0765, 21.456715, 21.273575, 21....","[[20.38853, 20.38853, 20.38853, 20.38853, 20.3...","[20.95322036743164, 20.893985748291016, 20.893...","[0.4799298346042633, 0.39916253089904785, 0.39..."
4,ackley,10,mcir,"[[20.974483489990234, 20.974483489990234, 20.9...","[[20.974483489990234, 20.974483489990234, 20.9...","[20.95499038696289, 20.95499038696289, 20.9549...","[0.4690623126870676, 0.4690623126870676, 0.469..."
...,...,...,...,...,...,...,...
165,watson,32,dual_annealing,"[[5393798.4407911245, 1104908.5937188813, 3621...","[[5393798.4407911245, 1104908.5937188813, 1104...","[37751606.73584604, 3791722.58588769, 2881944....","[62101331.395239055, 5219180.813468672, 538362..."
166,watson,32,lamcts,"[[4226062.596401818, 1520741.3544014867, 48223...","[[4226062.596401818, 1520741.3544014867, 48223...","[2854290.69828544, 647104.5967340438, 229400.0...","[3230807.167458508, 580186.7523225237, 323789...."
167,watson,32,mcir,"[[651574.6720559866, 651574.6720559866, 651574...","[[651574.6720559866, 651574.6720559866, 651574...","[588816.5247719139, 588816.5247719139, 588816....","[88753.42303854393, 88753.42303854393, 88753.4..."
168,watson,32,shgo,"[[], [], [], [], []]","[[], [], [], [], []]",[],[]


## Ablation studies

In [1]:
def get_info_from_ablation_filename(np_fname):
    splitted = np_fname.lower().split("_")
    func = splitted[0]
    dim = int(splitted[1])
    study_type = splitted[2]
    
    info = {
        "function": func,
        "dimension": dim,
        "study": study_type,
    }
    return info

In [75]:
ablation_directory = "/projects/mcmm/ablation_final/"

In [2]:
ablation_directory = "/home/ubuntu/s3/ablation_final/"

In [6]:
all_results = []
for filename in os.listdir(ablation_directory):
    assert filename.endswith("_Y.npy")
    result_dict = get_info_from_ablation_filename(filename)
    result_dict['obj_list'] = np.load(os.path.join(ablation_directory, filename))
    all_results.append(result_dict)

In [9]:
result_df = pd.DataFrame(all_results)
result_df['min_found_list'] = result_df['obj_list'].apply(process_list)
result_df = result_df.sort_values(["function", "dimension", "study"])
result_df.to_pickle("ablation_raw_result_df.pkl")
result_df

,function,dimension,study,obj_list,min_found_list
27,ackley,50,boxhigh,"[21.203022003173828, 21.203022003173828, 21.20...","[21.203022003173828, 21.203022003173828, 21.20..."
35,ackley,50,boxhigh,"[21.243709564208984, 21.243709564208984, 21.24...","[21.243709564208984, 21.243709564208984, 21.24..."
64,ackley,50,boxhigh,"[21.27104949951172, 21.27104949951172, 21.2710...","[21.27104949951172, 21.27104949951172, 21.2710..."
66,ackley,50,boxhigh,"[21.12533187866211, 21.12533187866211, 21.1253...","[21.12533187866211, 21.12533187866211, 21.1253..."
81,ackley,50,boxhigh,"[21.253238677978516, 21.253238677978516, 21.25...","[21.253238677978516, 21.253238677978516, 21.25..."
...,...,...,...,...,...
40,michalewicz,50,original,"[-4.662331581115723, -4.662331581115723, -4.66...","[-4.662331581115723, -4.662331581115723, -4.66..."
47,michalewicz,50,original,"[-4.188868522644043, -4.188868522644043, -4.18...","[-4.188868522644043, -4.188868522644043, -4.18..."
92,watson,32,local-opt,"[200290.4072673328, 76608.51186238528, 3356366...","[200290.4072673328, 76608.51186238528, 76608.5..."
24,watson,32,original,"[651574.6720559866, 651574.6720559866, 651574....","[651574.6720559866, 651574.6720559866, 651574...."


In [10]:
stat_df = result_df.groupby(['function', 'dimension', 'study']).agg(
    obj_lists=('obj_list', list),
    min_found_lists=('min_found_list', list)
).reset_index()
stat_df

,function,dimension,study,obj_lists,min_found_lists
0,ackley,50,boxhigh,"[[21.203022003173828, 21.203022003173828, 21.2...","[[21.203022003173828, 21.203022003173828, 21.2..."
1,ackley,50,boxlow,"[[21.341533660888672, 21.341533660888672, 21.3...","[[21.341533660888672, 21.341533660888672, 21.3..."
2,ackley,50,exphigh,"[[21.167007446289062, 21.167007446289062, 21.1...","[[21.167007446289062, 21.167007446289062, 21.1..."
3,ackley,50,explow,"[[21.39307403564453, 21.39307403564453, 21.393...","[[21.39307403564453, 21.39307403564453, 21.393..."
4,ackley,50,lbhigh,"[[21.402111053466797, 21.402111053466797, 21.4...","[[21.402111053466797, 21.402111053466797, 21.4..."
5,ackley,50,lblow,"[[21.17459487915039, 21.17459487915039, 21.174...","[[21.17459487915039, 21.17459487915039, 21.174..."
6,ackley,50,local-opt,"[[21.269081115722656, 21.308773040771484, 21.3...","[[21.269081115722656, 21.269081115722656, 21.2..."
7,ackley,50,no-learning,"[[21.0770263671875, 21.0770263671875, 21.07702...","[[21.0770263671875, 21.0770263671875, 21.07702..."
8,ackley,50,no-local-opt,"[[21.331951141357422, 21.331951141357422, 21.2...","[[21.331951141357422, 21.331951141357422, 21.2..."
9,ackley,50,nodeexphigh,"[[21.21166229248047, 21.21166229248047, 21.211...","[[21.21166229248047, 21.21166229248047, 21.211..."


In [13]:
applied_df = stat_df.apply(
    lambda row: calculate_mean_and_std_of_lists(row.min_found_lists), 
    axis='columns', 
    result_type='expand'
)
stat_df = pd.concat([stat_df, applied_df], axis='columns')

In [14]:
stat_df.to_pickle("ablation_result_df.pkl")
stat_df

,function,dimension,study,obj_lists,min_found_lists,avg_min_found,std_min_found
0,ackley,50,boxhigh,"[[21.203022003173828, 21.203022003173828, 21.2...","[[21.203022003173828, 21.203022003173828, 21.2...","[21.219270324707033, 21.219270324707033, 21.21...","[0.05813916062527396, 0.05813916062527396, 0.0..."
1,ackley,50,boxlow,"[[21.341533660888672, 21.341533660888672, 21.3...","[[21.341533660888672, 21.341533660888672, 21.3...","[21.35849609375, 21.35849609375, 21.3584960937...","[0.13442748777655683, 0.13442748777655683, 0.1..."
2,ackley,50,exphigh,"[[21.167007446289062, 21.167007446289062, 21.1...","[[21.167007446289062, 21.167007446289062, 21.1...","[21.25697479248047, 21.25697479248047, 21.2569...","[0.2141398681075259, 0.2141398681075259, 0.214..."
3,ackley,50,explow,"[[21.39307403564453, 21.39307403564453, 21.393...","[[21.39307403564453, 21.39307403564453, 21.393...","[21.277609252929686, 21.277609252929686, 21.27...","[0.1986563006285194, 0.1986563006285194, 0.198..."
4,ackley,50,lbhigh,"[[21.402111053466797, 21.402111053466797, 21.4...","[[21.402111053466797, 21.402111053466797, 21.4...","[21.202345275878905, 21.202345275878905, 21.20...","[0.19453338790692074, 0.19453338790692074, 0.1..."
5,ackley,50,lblow,"[[21.17459487915039, 21.17459487915039, 21.174...","[[21.17459487915039, 21.17459487915039, 21.174...","[21.208272552490236, 21.208272552490236, 21.20...","[0.15277253720062095, 0.15277253720062095, 0.1..."
6,ackley,50,local-opt,"[[21.269081115722656, 21.308773040771484, 21.3...","[[21.269081115722656, 21.269081115722656, 21.2...","[21.28687629699707, 21.20069999694824, 21.2000...","[0.16089645616233283, 0.15890722237908333, 0.1..."
7,ackley,50,no-learning,"[[21.0770263671875, 21.0770263671875, 21.07702...","[[21.0770263671875, 21.0770263671875, 21.07702...","[21.272610473632813, 21.272610473632813, 21.27...","[0.14408538028808102, 0.14408538028808102, 0.1..."
8,ackley,50,no-local-opt,"[[21.331951141357422, 21.331951141357422, 21.2...","[[21.331951141357422, 21.331951141357422, 21.2...","[21.316805267333983, 21.316805267333983, 21.14...","[0.1261281206985169, 0.1261281206985169, 0.239..."
9,ackley,50,nodeexphigh,"[[21.21166229248047, 21.21166229248047, 21.211...","[[21.21166229248047, 21.21166229248047, 21.211...","[21.114319610595704, 21.114319610595704, 21.11...","[0.08534272563178558, 0.08534272563178558, 0.0..."
